# Tensor Parallelism
In this session, we will explore tensor parallelism.


## 1. Intra-layer Model Parallelism
Tensor parallelism is an intra-layer model parallelism approach that **splits the model at the tensor level within a layer**. Inter-layer model parallelism is relatively intuitive, but for those encountering intra-layer parallelism for the first time, it may not be immediately clear how this is even possible.

![](../images/intra_layer.png)

The inner product operations that we commonly use have a property where splitting the matrices involved, performing the computations in parallel, and then summing or concatenating the results does not change the final output. Tensor parallelism leverages this property of inner product operations to parallelize models.

The terminology can be a bit confusing here: **intra-layer parallelism** is a broader concept that refers to all forms of parallelism that occur within a layer, while **tensor parallelism** is one specific method for implementing intra-layer parallelism.


## 2. Megatron-LM
Megatron-LM is an intra-layer model parallelism implementation released by NVIDIA, and it is currently one of the most important projects in large-scale model development.

<img src="../images/megatron_lm.jpeg" width=540>

### Column & Row Parallelism
Below are illustrations of column parallelism and row parallelism used in Megatron-LM.

- **Column parallelism** splits the model parameters (A) **vertically** into parts (A1, A2).
- **Row parallelism** splits the model parameters (A) **horizontally** into parts (A1, A2).

![](../images/intra_layer_2.png)

Let’s verify the results by coding them directly. First, consider the matrix multiplication result of tensor X and tensor A, as shown below.


In [ ]:
"""
src/non_parallelism.py
"""

import torch

X = torch.tensor(
    [
        [0, 1, 2, 3],
        [4, 5, 6, 7],
    ]
)

A = torch.tensor(
    [
        [10, 14],
        [11, 15],
        [12, 16],
        [13, 17],        
    ]
)

Y = X @ A

print(Y)

Column parallelism works by splitting the model parameters (A) **vertically**, performing the computation, and then **concatenating** the results. As shown in the figure, we replicate tensor X, split tensor A vertically, perform the computation, and then concatenate the outputs.


In [ ]:
"""
src/column_parallelism.py
"""

import torch

X = torch.tensor(
    [
        [0, 1, 2, 3],
        [4, 5, 6, 7],
    ]
)

A1 = torch.tensor(
    [
        [10],
        [11],
        [12],
        [13],        
    ]
)

A2 = torch.tensor(
    [
        [14],
        [15],
        [16],
        [17],        
    ]
)

Y1 = X @ A1
Y2 = X @ A2

print(Y1)
print(Y2)

Y = torch.cat([Y1, Y2], dim=1)
print(Y)

We can verify that the computation results are identical before and after parallelization.

Next, let’s look at **row parallelism**. Row parallelism works by splitting the model parameters (A) **horizontally** and then **summing** the computation results. As illustrated, we split both X and Y, perform the computations, and then add the results together.


In [ ]:
"""
src/row_parallelism.py
"""

import torch

X1 = torch.tensor(
    [
        [0, 1],
        [4, 5],
    ]
)

X2 = torch.tensor(
    [
        [2, 3],
        [6, 7],
    ]
)

A1 = torch.tensor(
    [
        [10, 14],
        [11, 15],      
    ]
)

A2 = torch.tensor(
    [
        [12, 16],
        [13, 17],        
    ]
)

Y1 = X1 @ A1
Y2 = X2 @ A2

print(Y1)
print(Y2)

Y = Y1 + Y2

print(Y)

We can confirm that the computation results are identical.

<br>

### Column Parallelism: $(D, D) \rightarrow (D, \frac{D}{n}) \times n$

As seen in the previous example, **column parallelism** works by **replicating the input tensor (X)**, splitting the model parameters (A) **vertically** into parts (A1, A2), performing the inner product, and then concatenating the results.

<br>

![](../images/column_parallel.png)

<br>

In Megatron-LM, the **partitioned parameters (A1, A2) are placed on different devices to parallelize the model**. As a result, the matrix multiplication is performed simultaneously across multiple GPUs, which requires distributed programming to coordinate the computation. For column parallelism, **Broadcast** and **All-gather** operations are used.

- **Broadcast** is used to send the same input to different GPUs.
- **All-gather** is used to collect the results of the matrix multiplications.


In [ ]:
"""
Note: ColumnParallelLinear in megatron-lm/megatron/mpu/layers.py
"""

def forward(self, input_):
    bias = self.bias if not self.skip_bias_add else None

    # Set up backprop all-reduce.
    input_parallel = copy_to_tensor_model_parallel_region(input_)

    # Matrix multiply.
    output_parallel = F.linear(input_parallel, self.weight, bias)

    if self.gather_output:
        output = gather_from_tensor_model_parallel_region(output_parallel)
    else:
        output = output_parallel
    
    output_bias = self.bias if self.skip_bias_add else None
    return output, output_bias

### Row Parallelism: $(D, D) \rightarrow (\frac{D}{n}, D) \times n$

Row parallelism works by **splitting the input tensor (X)** and splitting the model parameters (A) **horizontally** into parts (A1, A2), performing the inner product, and then **summing** the results.

<br>

![](../images/row_parallelism.png)

<br>

Similarly, executing row parallelism across multiple GPUs requires distributed programming. For row parallelism, **Scatter** and **All-reduce** operations are used.

- **Scatter** is used to split and distribute the input across different GPUs.
- **All-reduce** is used to sum the results of the matrix multiplications.


In [ ]:
"""
Note: RowParallelLinear in megatron-lm/megatron/mpu/layers.py
"""

def forward(self, input_):
    # Set up backprop all-reduce.
    if self.input_is_parallel:
        input_parallel = input_
    else:
        input_parallel = scatter_to_tensor_model_parallel_region(input_)
    
    # Matrix multiply.
    output_parallel = F.linear(input_parallel, self.weight)
    
    # All-reduce across all the partitions.
    output_ = reduce_from_tensor_model_parallel_region(output_parallel)
    
    if not self.skip_bias_add:
        output = output_ + self.bias if self.bias is not None else output_
        output_bias = None
    else:
        output = output_
        output_bias = self.bias
    return output, output_bias

### Transformer Block

Now that we understand column and row parallelism, let’s take a closer look at how a Transformer can be parallelized. A typical Transformer block that we are familiar with is structured as shown below. In Megatron-LM, layers with very small parameter sizes—such as **LayerNorm**—have their parameters replicated across all devices. The other layers (Attention and MLP), excluding LayerNorm, are parallelized using column and row parallelism as described above.

![](../images/megatron_block.png)

<br>

### MLP Layer

Let’s start by examining the MLP layer. The MLP layer consists of the following sequence:  
`Linear1` → `GeLU` → `Linear2` → `Dropout`.

<br>

![](../images/megatron_mlp.png)

<br>


In [ ]:
"""
Note: transformers/models/gpt_neo/modeling_gpt_neo.py
"""

import torch.nn as nn


class GPTNeoMLP(nn.Module):
    def __init__(self, intermediate_size, config):  # in MLP: intermediate_size= 4 * hidden_size
        super().__init__()
        embed_dim = config.hidden_size
        self.c_fc = nn.Linear(embed_dim, intermediate_size)
        self.c_proj = nn.Linear(intermediate_size, embed_dim)
        self.act = ACT2FN[config.activation_function]
        self.dropout = nn.Dropout(config.resid_dropout)

    def forward(self, hidden_states):
        hidden_states = self.c_fc(hidden_states)
        hidden_states = self.act(hidden_states)
        hidden_states = self.c_proj(hidden_states)
        hidden_states = self.dropout(hidden_states)
        return hidden_states

Here, the **first Linear layer uses column parallelism**, and the **second Linear layer uses row parallelism**.

<br>

![](../images/megatron_mlp_2.png)

<br>

There are two reasons why column–row parallelism is applied in this order in the MLP layer.

- The first reason is that it allows us to **skip the `All-gather` and `Scatter` operations**.

<br>

![](../images/megatron_mlp_3.png)

<br>

The computation in the green region on the left is the inner product between the input data X and the weight matrix W that has been parallelized across devices. Then, in the red region, these results are `All-gather`ed and concatenated, and subsequently `Scatter`ed again. An interesting observation here is that since the concatenated tensor is split again, the result is identical to the state before concatenation. Therefore, the values in the green region on the right are the same as those in the green region on the left.

As a result, the red region (`All-gather` → `Scatter`) can be omitted, leading to a significant performance gain.

This is a unique phenomenon that occurs **only when parallelization is applied in a column–row order**. If other combinations such as column–column, row–column, or row–row parallelism are used, the communication between the two Linear layers cannot be skipped.

<br>

![](../images/megatron_mlp_4.png)

<br>

The technique for skipping `All-gather` and `Scatter` is implemented in Megatron-LM via the parameters `input_is_parallel` and `gather_output`.


In [ ]:
"""
Reference: ColumnParallelLinear in megatron-lm/megatron/mpu/layers.py
"""

def forward(self, input_):
    bias = self.bias if not self.skip_bias_add else None

    # Set up backprop all-reduce.
    input_parallel = copy_to_tensor_model_parallel_region(input_)

    # Matrix multiply.
    output_parallel = F.linear(input_parallel, self.weight, bias)

    # Set gather_output to False to return the output in a parallelized form.
    if self.gather_output:
        output = gather_from_tensor_model_parallel_region(output_parallel)
    else:
        output = output_parallel

    output_bias = self.bias if self.skip_bias_add else None
    return output, output_bias


"""
Reference: RowParallelLinear in megatron-lm/megatron/mpu/layers.py
"""

def forward(self, input_):
    # Set up backprop all-reduce.

    # Set input_is_parallel to True to receive the input in a parallelized form.
    if self.input_is_parallel:
        input_parallel = input_
    else:
        input_parallel = scatter_to_tensor_model_parallel_region(input_)
    
    # Matrix multiply.
    output_parallel = F.linear(input_parallel, self.weight)
    
    # All-reduce across all partitions.
    output_ = reduce_from_tensor_model_parallel_region(output_parallel)
    
    if not self.skip_bias_add:
        output = output_ + self.bias if self.bias is not None else output_
        output_bias = None
    else:
        output = output_
        output_bias = self.bias
    return output, output_bias


- The second reason for using **column–row parallelism** is that, in order to skip `Scatter` and `All-gather`, the **GeLU operation must be executed in a parallelized form**.

<br>

![](../images/megatron_mlp_5.png)

<br>

The figure above shows a case where the GeLU operation is inserted between the two Linear layers **without skipping** `Scatter` and `All-gather`. If we implement the model in a way that skips these two operations, then the GeLU operation must be executed independently on each device, as shown below.

<br>

![](../images/megatron_mlp_6.png)

<br>

However, for GeLU to be parallelized across different devices in this way, the output of the GeLU computed in parallel must be identical to the output of the GeLU computed in a non-parallelized manner. In other words, the following equations must hold. (The symbol $\circledcirc$ denotes concatenation.)

<br>

$$
\text{Row Parallelism: } \mathrm{GeLU}(XW_1 + XW_2) = \mathrm{GeLU}(XW_1) + \mathrm{GeLU}(XW_2)
$$

<br>

$$
\text{Column Parallelism: } \mathrm{GeLU}(XW_1 \circledcirc XW_2) = \mathrm{GeLU}(XW_1) \circledcirc \mathrm{GeLU}(XW_2)
$$

<br>

The problem is that these equations **only hold for column parallelism**, and **do not hold for row parallelism**.

<br>

$$
\text{Row Parallelism: } \mathrm{GeLU}(XW_1 + XW_2) \neq \mathrm{GeLU}(XW_1) + \mathrm{GeLU}(XW_2)
$$

<br>

Let’s verify this by implementing it in code.


In [ ]:
"""
src/megatron_mlp_gelu.py
"""

import torch
from torch.nn.functional import gelu


w = torch.randn(6, 6)
x = torch.randn(6, 6)


class RowParallelLinear(torch.nn.Module):
    def __init__(self):
        super(RowParallelLinear, self).__init__()
        chunked = torch.chunk(w, 2, dim=0)

        # row parallelized parameters
        self.w1 = chunked[0]  # [3, 6]
        self.w2 = chunked[1]  # [3, 6]

    def forward(self, x):
        # GeLU(X1A1 + X2A2) != GeLU(X1A1) + GeLU(X2A2)
        x1, x2 = torch.chunk(x, 2, dim=1)

        # parallel output
        y1 = gelu(x1 @ self.w1) + gelu(x2 @ self.w2)

        # non-parallel output
        y2 = gelu(x1 @ self.w1 + x2 @ self.w2)

        return torch.all(y1 == y2)


class ColumnParallelLinear(torch.nn.Module):
    def __init__(self):
        super(ColumnParallelLinear, self).__init__()
        chunked = torch.chunk(w, 2, dim=1)

        # column parallelized parameters
        self.w1 = chunked[0]  # [6, 3]
        self.w2 = chunked[1]  # [6, 3]

    def forward(self, x):
        # GeLU(X1A1 cat X2A2) == GeLU(X1A1) cat GeLU(X2A2)

        # parallel output
        y1 = torch.cat([gelu(x @ self.w1), gelu(x @ self.w2)], dim=1)

        # non-parallel output
        y2 = gelu(torch.cat([(x @ self.w1), (x @ self.w2)], dim=1))

        return torch.all(y1 == y2)


# Row Parallelism
print("Is GeLU in RowParallelLinear same with non-parallel = ", end="")
print(RowParallelLinear()(x).item())

# Column Parallelism
print("Is GeLU in ColumnParallelLinear same with non-parallel = ", end="")
print(ColumnParallelLinear()(x).item())

Therefore, in order to parallelize the GeLU operation, the Linear layer preceding GeLU must be parallelized in the **column direction**. As a result, applying parallelism in a **column–row order** is the most efficient approach.


<br>

### Multi-head Attention Layer

Next, let’s examine the Multi-head Attention layer. The Multi-head Attention layer follows the sequence:  
`Linear1` → `Split heads` → `Scaled Dot-Product Attention` → `Concat (Merge) heads` → `Linear2` → `Dropout`.

![](../images/multi_head_attention.png)



In [ ]:
"""
Reference: transformers/models/gpt_neo/modeling_gpt_neo.py
"""

class GPTNeoSelfAttention(nn.Module):
    def __init__(self, config, attention_type):
        super().__init__()
        self.attn_dropout = nn.Dropout(config.attention_dropout)
        self.resid_dropout = nn.Dropout(config.resid_dropout)

        self.embed_dim = config.hidden_size
        self.num_heads = config.num_heads
        self.head_dim = self.embed_dim // self.num_heads
        if self.head_dim * self.num_heads != self.embed_dim:
            raise ValueError(
                f"embed_dim must be divisible by num_heads (got `embed_dim`: {self.embed_dim} and `num_heads`: {self.num_heads})."
            )

        self.k_proj = nn.Linear(self.embed_dim, self.embed_dim, bias=False)
        self.v_proj = nn.Linear(self.embed_dim, self.embed_dim, bias=False)
        self.q_proj = nn.Linear(self.embed_dim, self.embed_dim, bias=False)
        self.out_proj = nn.Linear(self.embed_dim, self.embed_dim, bias=True)

    def forward(
        self,
        hidden_states,
        attention_mask=None,
        layer_past=None,
        head_mask=None,
        use_cache=False,
        output_attentions=False,
    ):
        # 1. linear projection
        query = self.q_proj(hidden_states)
        key = self.k_proj(hidden_states)
        value = self.v_proj(hidden_states)
        
        # 2. split heads
        query = self._split_heads(query, self.num_heads, self.head_dim)
        key = self._split_heads(key, self.num_heads, self.head_dim)
        value = self._split_heads(value, self.num_heads, self.head_dim)

        # 3. scale dot product attention
        attn_output, attn_weights = self._attn(query, key, value, attention_mask, head_mask)

        # 4. concat (merge) heads
        attn_output = self._merge_heads(attn_output, self.num_heads, self.head_dim)
        
        # 5. linear projection
        attn_output = self.out_proj(attn_output)
        
        # 6. dropout
        attn_output = self.resid_dropout(attn_output)

        return outputs

![](../images/megatron_attention.jpeg)

<br>

Megatron-LM parallelizes the **Q, K, V linear projections** and the **output projection** in the Attention layer. Similarly, the Q, K, V linear projection layers are handled using **column parallelism**, while the output projection layer is handled using **row parallelism**, forming a **column–row pattern**. 

By doing so, the Attention layer—just like the MLP layer—can **skip the `Scatter` and `All-gather` operations**, resulting in more efficient execution.

<br>


### Vocab Parallel Embedding

Megatron-LM also parallelizes the word embedding layer. A unique aspect is that the parallelization is done along the **vocabulary size dimension**. For example, if we have a word embedding matrix with a vocabulary size of 50,000, the matrix has a shape of `(50,000, embedding_dim)`. Megatron-LM parallelizes this matrix along the vocabulary dimension. This distinctive parallelization technique is called **Vocab Parallel Embedding**.

![](../images/vpe_1.png)

<br>

The figure above shows word embedding without parallelization. When a sequence of length 6 is given as input, an input tensor of shape `[6, embedding_dim]` is produced.

<br>

![](../images/vpe_2.png)

The figure above illustrates how vocab parallel embedding works. The original embedding matrix is split into two halves: one embedding matrix responsible for tokens from index 0 to 24,999, and another responsible for tokens from index 25,000 to 50,000. When input data is provided, **tokens that fall outside the range covered by a given matrix are masked out**. Then, **the vectors corresponding to the masked positions are initialized to all zeros**, and finally, the two matrices are **summed together**. This results in a complete input tensor that contains the correct embedding vectors for all words.


In [ ]:
"""
Reference: VocabParallelEmbedding in megatron-lm/megatron/mpu/layers.py
"""

def forward(self, input_):
    if self.tensor_model_parallel_size > 1:
        # Build the mask.
        input_mask = (input_ < self.vocab_start_index) | \
                     (input_ >= self.vocab_end_index)

        # Mask the input.
        masked_input = input_.clone() - self.vocab_start_index
        masked_input[input_mask] = 0

    else:
        masked_input = input_
        # Get the embeddings.
    
    output_parallel = F.embedding(masked_input, self.weight,
                                  self.padding_idx, self.max_norm,
                                  self.norm_type, self.scale_grad_by_freq,
                                  self.sparse)

    # Mask the output embedding.
    if self.tensor_model_parallel_size > 1:
        output_parallel[input_mask, :] = 0.0
    
    # Reduce across all the model parallel GPUs.
    output = reduce_from_tensor_model_parallel_region(output_parallel)
    return output

However, a problem arises here. Tensor parallelism requires the vocabulary size to be divisible by the number of GPUs used for parallelization. Since 52,527 is not an even number, it cannot be evenly split across two GPUs. To address this, unused tokens are added to the word embedding matrix to make the vocabulary size divisible. This adjusted size is called the **padded vocab size**.

In Megatron-LM, the vocabulary size can be adjusted using the `make-vocab-size-divisible-by` argument, which ensures that the vocabulary size becomes a multiple of the specified value.

In conclusion, by applying **vocab parallel embedding**, Megatron-LM can further improve memory efficiency.


<br>

### Vocab Parallel Cross Entropy

Tasks such as **causal language modeling** in GPT-2 or **masked language modeling** in BERT generate natural language tokens as the final output. Therefore, after passing through the final Transformer layer, the model’s output shape expands to `(batch_size, sequence_length, vocab_size)`. (This does not apply to tasks such as classification or tagging.)

<br>

![](../images/lm_head.png)

<br>

In this case, if **input and output embeddings are tied** (weight tying), the word embedding matrix is reused instead of initializing new parameters for the Linear layer used in the Language Modeling Head (hereafter referred to as the **LM Head**). In most publicly released models such as BERT, GPT-2, and GPT-Neo, the output embeddings (LM Head) are tied to the input embeddings.


In [ ]:
"""
Reference: transformers/models/gpt_neo/modeling_gpt_neo.py
"""

class GPTNeoForCausalLM(GPTNeoPreTrainedModel):
    _keys_to_ignore_on_load_missing = [
        r"h\.\d+\.attn\.masked_bias",
        r"lm_head\.weight",
        r"h\.\d+\.attn\.attention\.bias",
    ]
    _keys_to_ignore_on_save = [r"lm_head.weight"]
    # 3. Therefore, the `lm_head.weight` parameter is not loaded or saved.
    # There is no need to store or load the same tensor twice.

    def __init__(self, config):
        super().__init__(config)
        self.transformer = GPTNeoModel(config)
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)
        # 1. At first glance, it appears as if the nn.Linear layer parameters
        #    are newly allocated and used.

        self.init_weights()
        # 2. However, when this method is called, the input and output embeddings
        #    (lm head) are tied.
        #    At this point, the word embedding matrix weights are copied to the
        #    nn.Linear layer’s weights.
        #    This copy is not a deep copy, but a shallow copy
        #    (the values are shared, not just the reference).
        #    Therefore, `lm_head.weight` is a single tensor that resides in the
        #    same memory address space as the word embedding.

    def get_output_embeddings(self):
        return self.lm_head

    def set_output_embeddings(self, new_embeddings):
        self.lm_head = new_embeddings


In [ ]:
"""
Reference: transformers/modeling_utils.py
"""

def init_weights(self):
    """
    If needed prunes and maybe initializes weights.
    """
    # Prune heads if needed
    if self.config.pruned_heads:
        self.prune_heads(self.config.pruned_heads)

    if _init_weights:
        # Initialize weights
        self.apply(self._init_weights)

        # For models that support weight tying, calling this method
        # automatically ties the input embeddings and the output embeddings
        # (= lm head).
        self.tie_weights()


def tie_weights(self):
    """
    Tie the weights between the input embeddings and the output embeddings.
    If the :obj:`torchscript` flag is set in the configuration, can't handle parameter sharing so we are cloning
    the weights instead.
    """
    output_embeddings = self.get_output_embeddings()
    if output_embeddings is not None and self.config.tie_word_embeddings:
        self._tie_or_clone_weights(output_embeddings, self.get_input_embeddings())
        # When this method is called, the output embeddings (lm head)
        # are tied to the input embeddings.

    if self.config.is_encoder_decoder and self.config.tie_encoder_decoder:
        if hasattr(self, self.base_model_prefix):
            self = getattr(self, self.base_model_prefix)
        self._tie_encoder_decoder_weights(self.encoder, self.decoder, self.base_model_prefix)

    for module in self.modules():
        if hasattr(module, "_tie_weights"):
            module._tie_weights()


However, a problem arises here. In general, when computing the loss between the logits produced by the LM Head and the target data, the following process takes place.

<br>

![](../images/vpce_1.png)

<br>

However, since Megatron-LM uses **vocab parallel embedding**, the embedding layer is split across multiple devices. As a result, when weight tying is applied, the **output embeddings (LM Head) are also split across multiple devices**. Consequently, the size of the logits produced by the model corresponds to a partitioned vocabulary size.

<br>

![](../images/vpce_2.png)

<br>

As shown in the figure above, if the vocabulary size is 50,000, the output logits would normally have the shape `(batch_size, length, 50,000)`. But if the vocabulary is split across two devices, each device produces logits of shape `(batch_size, length, 25,000)`. The logits on each device contain different values. **These are referred to as Parallel LM Logits.**  

In this situation, how should we compute the loss against the target sentence? The target data contains token indices from 0 to 49,999, whereas each device’s logits cover only half of that range.

<br>

![](../images/vpce_3.png)

<br>

In this case, we must use a special loss function called **vocab parallel cross entropy**, rather than the standard cross entropy. The computation of vocab parallel cross entropy proceeds as illustrated above. From the computed logits, only the portion of the vocabulary covered by the current device is retained, and the rest is masked out when computing the loss. Then, the losses computed on each device are **All-reduced** to obtain the final loss value.


In [ ]:
"""
Reference: _VocabParallelCrossEntropy in megatron-lm/megatron/mpu/cross_entropy.py
"""

@staticmethod
def forward(ctx, vocab_parallel_logits, target):
    # Compute the maximum value along the vocab dimension across all GPUs.
    logits_max = torch.max(vocab_parallel_logits, dim=-1)[0]
    torch.distributed.all_reduce(logits_max,
                                 op=torch.distributed.ReduceOp.MAX,
                                 group=get_tensor_model_parallel_group())

    # Subtract the maximum value.
    vocab_parallel_logits.sub_(logits_max.unsqueeze(dim=-1))

    # Get the partitioned vocab indices.
    get_vocab_range = VocabUtility.vocab_range_from_per_partition_vocab_size
    partition_vocab_size = vocab_parallel_logits.size()[-1]
    rank = get_tensor_model_parallel_rank()
    world_size = get_tensor_model_parallel_world_size()
    vocab_start_index, vocab_end_index = get_vocab_range(
        partition_vocab_size, rank, world_size)

    # Create a mask for valid vocab IDs (1 means the token should be masked).
    target_mask = (target < vocab_start_index) | (target >= vocab_end_index)
    masked_target = target.clone() - vocab_start_index
    masked_target[target_mask] = 0

    # Extract predicted logits = logits[target].
    # For simplicity, convert logits into a 2-D tensor of shape
    # [*, partition_vocab_size] and target into a 1-D tensor of shape [*].
    logits_2d = vocab_parallel_logits.view(-1, partition_vocab_size)
    masked_target_1d = masked_target.view(-1)
    arange_1d = torch.arange(start=0, end=logits_2d.size()[0],
                                 device=logits_2d.device)
    predicted_logits_1d = logits_2d[arange_1d, masked_target_1d]
    predicted_logits_1d = predicted_logits_1d.clone().contiguous()
    predicted_logits = predicted_logits_1d.view_as(target)
    predicted_logits[target_mask] = 0.0
    
    # All-reduce is required to collect chunks from other GPUs.
    torch.distributed.all_reduce(predicted_logits,
                                 op=torch.distributed.ReduceOp.SUM,
                                 group=get_tensor_model_parallel_group())

    # Compute the sum of exp(logits) along the vocab dimension across all GPUs.
    exp_logits = vocab_parallel_logits
    torch.exp(vocab_parallel_logits, out=exp_logits)
    sum_exp_logits = exp_logits.sum(dim=-1)
    torch.distributed.all_reduce(sum_exp_logits,
                                 op=torch.distributed.ReduceOp.SUM,
                                 group=get_tensor_model_parallel_group())

    # Loss = log(sum(exp(logits))) - predicted_logit.
    loss = torch.log(sum_exp_logits) - predicted_logits

    # Store softmax, target mask, and masked target for the backward pass.
    exp_logits.div_(sum_exp_logits.unsqueeze(dim=-1))
    ctx.save_for_backward(exp_logits, target_mask, masked_target_1d)

    return loss


### Training a Model with Megatron-LM

Let’s try training a model using Megatron-LM. Unlike Hugging Face’s `transformers`, Megatron-LM is not a framework that is typically used at the code level. Instead, it provides a well-structured codebase that is used to build and train models. Therefore, we will proceed by first cloning the repository.


In [ ]:
# If git and wget are not installed, install them using the following command.
!apt update && apt install git wget -y


In [ ]:
# Clone the Megatron-LM repository.
!git clone https://github.com/NVIDIA/Megatron-LM


In [ ]:
%cd Megatron-LM

Next, let’s install a few required packages. Megatron-LM includes functionality that uses `nltk` to split data into sentences during preprocessing. Although we won’t be using this feature here, an error will occur if `nltk` is not installed, so we will install it.


In [ ]:
!pip install nltk

Megatron-LM also uses the `pybind11` and `apex` packages. Let’s install them as well.  
(The CUDA compilation can take quite a while, so please be patient.)


Megatron-LM also uses the `pybind11` and `apex` packages. We will install them as well.  
(The CUDA compilation takes quite a long time, so please be patient.)


In [ ]:
!pip install pybind11

In [ ]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
%cd 

Next, let’s create the dataset. When pre-training a model with Megatron-LM, you only need to create a simple `jsonl` file where each line follows a structure like `{"text": "sample text"}`. For fine-tuning, however, the dataset must be constructed according to the specific task.

Since this tutorial focuses only on **pre-training**, we will not cover fine-tuning here. If you need fine-tuning, please refer to the Megatron-LM GitHub repository.


In [ ]:
"""
src/megatron_datasets.py
"""

import json
import os
from datasets import load_dataset

train_samples, min_length = 10000, 512
filename = "megatron_datasets.jsonl"
curr_num_datasets = 0

if os.path.exists(filename):
    os.remove(filename)

datasets = load_dataset("wikitext", "wikitext-103-raw-v1")
datasets = datasets.data["train"]["text"]
dataset_fp_write = open(filename, mode="w", encoding="utf-8")

for sample in datasets:
    sample = sample.as_py()

    if len(sample) >= min_length:
        line = json.dumps(
            {"text": sample},
            ensure_ascii=False,
        )

        dataset_fp_write.write(line + "\n")
        curr_num_datasets += 1

        # Since this is a tutorial, we will create only a small amount of data.
        if curr_num_datasets >= train_samples:
            break

dataset_fp_read = open(filename, mode="r", encoding="utf-8")
dataset_read = dataset_fp_read.read().splitlines()[:3]

# Check the structure of the data.
for sample in dataset_read:
    print(sample, end="\n\n")


In [ ]:
%cd Large-scale-lm-eng-tutorial/

In [ ]:
!python src/ch6_tensor_parallelism/megatron_datasets.py

Download the vocabulary to be used for tokenization

In [ ]:
!wget https://huggingface.co/gpt2/raw/main/vocab.json
!wget https://huggingface.co/gpt2/raw/main/merges.txt

In [ ]:
%ls

Now we preprocess the dataset. The preprocessing performed here includes both tokenization and binarization. Megatron-LM’s preprocessing code is copied from Fairseq’s indexed dataset implementation. Fairseq provides three main preprocessing modes—lazy, cached, and mmap. Before proceeding, we will briefly explain these preprocessing methods.

<b>1) Lazy

The lazy mode loads the required data from disk into memory at every step. In other words, whenever the __getitem__() method of the Dataset class is called, the data is loaded into memory by accessing the specified file path. However, because disk I/O is performed through the file buffer at every step, this approach can be relatively slow.

In [ ]:
"""
Reference: fairseq/fairseq/data/indexed_dataset.py
"""


from typing import Union
import numpy as np


def __getitem__(self, idx: Union[int, slice]) -> np.ndarray:
    if not self.data_file:
        # Load file buffer
        self.read_data(self.path)

    if isinstance(idx, int):
        # Validate index
        self.check_index(idx)

        # Compute the size of the tensor to load
        tensor_size = self.sizes[self.dim_offsets[idx] : self.dim_offsets[idx + 1]]

        # Allocate empty memory space to store the tensor
        array = np.empty(tensor_size, dtype=self.dtype)

        # Set the disk address to read from based on the offset
        self.data_file.seek(self.data_offsets[idx] * self.element_size)

        # Load data from disk into memory (file I/O)
        self.data_file.readinto(array)
        return array


#### 2) Cached
`cached` prefetches all data before training and keeps it in memory. Since disk access is not required for data loading during training, it is generally faster than the other methods. However, because memory capacity is limited, it is difficult to use this method when the dataset size is very large.


In [ ]:
"""
Reference: fairseq/fairseq/data/indexed_dataset.py
Comments were added manually by me.
"""


from typing import List


def prefetch(self, indices: List[int]) -> None:
    if all(i in self.cache_index for i in indices):
        # If all data has already been cached, exit the method
        return

    if not self.data_file:
        # If the file buffer is not loaded, load the file buffer
        self.read_data(self.path)

    # Sort indices to calculate the total contiguous memory size
    indices = sorted(set(indices))

    total_size = 0
    for i in indices:
        total_size += self.data_offsets[i + 1] - self.data_offsets[i]

    # Allocate the total memory space to be used as cache
    self.cache = np.empty(
        total_size,
        dtype=self.dtype,
    )

    self.cache_index.clear()
    ptx = 0

    for i in indices:
        # Store the starting position of each array
        self.cache_index[i] = ptx

        # Compute the data size from offsets and assign memory space for the current sample
        size = self.data_offsets[i + 1] - self.data_offsets[i]
        array = self.cache[ptx : ptx + size]

        # Set the disk address to read from based on the offset
        self.data_file.seek(self.data_offsets[i] * self.element_size)

        # Write the current sample into the allocated memory
        self.data_file.readinto(array)
        ptx += size

    if self.data_file:
        # All data has been loaded from the file buffer, so close the buffer and release the reference
        self.data_file.close()
        self.data_file = None


In [ ]:
"""
Reference: fairseq/fairseq/data/indexed_dataset.py
Comments were added manually by me.
"""

def __getitem__(self, idx: Union[int, tuple]) -> Union[np.ndarray, List]:
    if isinstance(idx, int):
        # Validate index
        self.check_index(idx)

        # Compute tensor size
        tensor_size = self.sizes[self.dim_offsets[idx] : self.dim_offsets[idx + 1]]

        # Allocate memory space
        array = np.empty(tensor_size, dtype=self.dtype)

        # Load prefetched data (no file I/O occurs)
        ptx = self.cache_index[idx]

        # Copy prefetched data from cache into the allocated memory space
        np.copyto(array, self.cache[ptx : ptx + array.size])
        return array

    elif isinstance(idx, slice):
        return [self[i] for i in range(*idx.indices(len(self)))]


#### 3) Mmap
`mmap`, like `lazy`, loads only the required amount of data into memory at each step, but it uses a Memory Map instead of a File Buffer. Unlike a File Buffer, a Memory Map maps the file address into the virtual memory allocated to the current process, allowing the data to be accessed as if it already resides in memory. It does not perform direct disk I/O, loads data in page-sized units (4KB), and since all operations effectively occur in memory, it generally provides faster processing speed compared to a File Buffer.


In [ ]:
"""
Reference: fairseq/fairseq/data/indexed_dataset.py
Comments were added manually by me.
"""

def __init__(self, path: str):
    with open(path, "rb") as stream:
        # 1. Load magic string
        # The magic string is used to identify the format of the stored data structure.
        # Whether it is lazy, mmap, etc... (cached uses the same value as lazy)
        magic_test = stream.read(9)
        assert magic_test == self._HDR_MAGIC, (
            "Index file doesn't match expected format. "
            "Please check your configuration file."
        )
        
        # 2. Load version (little endian unsigned long long)
        # Looking at the code, the version is always written as 1, so this seems to be a variable with little practical meaning.
        # b'\x01\x00\x00\x00\x00\x00\x00\x00'
        version = struct.unpack("<Q", stream.read(8))
        assert (1,) == version

        # 3. Load data type (little endian unsigned char)
        (dtype_code,) = struct.unpack("<B", stream.read(1))
        self._dtype = _code_to_dtype[dtype_code]
        self._dtype_size = self._dtype().itemsize

        # 4. Load total dataset length (little endian unsigned long long)
        self._len = struct.unpack("<Q", stream.read(8))[0]

        # 5. Load total number of samples (little endian unsigned long long)
        self._doc_count = struct.unpack("<Q", stream.read(8))[0]
        offset = stream.tell()

    # 6. Perform cache warmup
    _warmup_mmap_file(path)

    # 7. Create memory-mapped array
    self._bin_buffer_mmap = np.memmap(path, mode="r", order="C")
    self._bin_buffer = memoryview(self._bin_buffer_mmap)

    # 8. Load sample sizes into a memory-mapped array
    self._sizes = np.frombuffer(
        self._bin_buffer, dtype=np.int32, count=self._len, offset=offset
    )

    # 9. Load data pointers (offsets) into a memory-mapped array
    self._pointers = np.frombuffer(
        self._bin_buffer,
        dtype=np.int64,
        count=self._len,
        offset=offset + self._sizes.nbytes,
    )

    # 10. Load document indices into a memory-mapped array
    self._doc_idx = np.frombuffer(
        self._bin_buffer,
        dtype=np.int64,
        count=self._doc_count,
        offset=offset + self._sizes.nbytes + self._pointers.nbytes,
    )


In [ ]:
"""
Reference: fairseq/fairseq/data/indexed_dataset.py
Comments were added manually by me.
"""

from typing import Union
import numpy as np

def __getitem__(self, idx: Union[int, slice]) -> np.ndarray:
    if not self.data_file:
        # If the index file is not loaded, load it
        self.read_data(self.path)

    if isinstance(idx, int):
        # Validate index
        self.check_index(idx)

        # Compute tensor size
        tensor_size = self.sizes[self.dim_offsets[idx] : self.dim_offsets[idx + 1]]

        # Allocate memory space
        array = np.empty(tensor_size, dtype=self.dtype)

        # Set the virtual memory address of the data to read based on the offset
        self.data_file.seek(self.data_offsets[idx] * self.element_size)

        # Load data into memory
        self.data_file.readinto(array)
        return array

    elif isinstance(idx, slice):
        start, stop, step = idx.indices(len(self))
        if step != 1:
            # When using slicing, it must be contiguous
            raise ValueError("Slices into indexed_dataset must be contiguous")

        # Compute the list of tensor sizes and their total sum
        sizes = self.sizes[self.dim_offsets[start] : self.dim_offsets[stop]]
        total_size = sum(sizes)

        # Allocate the required amount of memory
        array = np.empty(total_size, dtype=self.dtype)

        # Set the virtual memory address of the data to read based on the offset
        self.data_file.seek(self.data_offsets[start] * self.element_size)
        self.data_file.readinto(array)

        # Split into multiple samples based on tensor sizes
        offsets = list(accumulate(sizes))
        sentences = np.split(array, offsets[:-1])
        return sentences


Now we perform dataset preprocessing. I will use the `mmap` method for preprocessing. 

At this point, you may notice an option called `append-eod`. Megatron-LM concatenates all data during pre-training in order to avoid creating padding. For example, if there are samples such as `{"text": "I am a boy."}` and `{"text": "You are so lucky"}`, during pre-training all samples are concatenated like `input = "I am a boy. You are so lucky ..."` and then the data is split according to the user-defined length (e.g. 2048) for training. 

However, if all samples are concatenated into a single string, the boundaries between samples can be lost, which may cause issues. When the `append-eod` option is enabled, an `end of document` token is inserted between samples to distinguish them. In the case of GPT2, the `eod` token is set to the `eos` token.


In [ ]:
!python tools/preprocess_data.py \
       --input megatron_datasets.jsonl \
       --output-prefix my-gpt2 \
       --vocab vocab.json \
       --dataset-impl mmap \
       --tokenizer-type GPT2BPETokenizer \
       --merge-file merges.txt \
       --append-eod

Dataset preprocessing has been completed. Let’s take a look at the data.

- my-gpt2_text_document.bin
- my-gpt2_text_document.idx

Files like these have been created. The `idx` file stores metadata such as data offsets and locations, while the `bin` file contains the actual tokenized data.


In [ ]:
%ls


Now we will start model training.


In [ ]:
# For now, we will use only Tensor parallelism.
# Data parallelism and Pipeline parallelism will be used in the Multi-dimensional Parallelism session. :)
# We will train for only 1000 steps. Please set a larger number for actual training.


!python -m torch.distributed.launch \
                  --nproc_per_node "4" \
                  --nnodes "1" \
                  --node_rank "0" \
                  --master_addr "localhost" \
                  --master_port "6000" \
                  ./pretrain_gpt.py \
                  --num-layers "24" \
                  --hidden-size "1024" \
                  --num-attention-heads "16" \
                  --seq-length "1024" \
                  --max-position-embeddings "1024" \
                  --micro-batch-size "4" \
                  --global-batch-size "8" \
                  --lr "0.00015" \
                  --train-iters "1000" \
                  --lr-decay-iters "300" \
                  --lr-decay-style cosine \
                  --vocab-file "vocab.json" \
                  --merge-file "merges.txt" \
                  --lr-warmup-fraction ".01" \
                  --fp16 \
                  --log-interval "10" \
                  --save-interval "500" \
                  --eval-interval "100" \
                  --eval-iters 10 \
                  --activations-checkpoint-method "uniform" \
                  --save "checkpoints/gpt2_345m" \
                  --load "checkpoints/gpt2_345m" \
                  --data-path "my-gpt2_text_document" \
                  --tensor-model-parallel-size "4" \
                  --pipeline-model-parallel-size "1" \
                  --DDP-impl "torch"

# Megatron-LM provides many more options in addition to the ones set above.
# It is difficult to explain all options, so please refer to the link below.
# https://github.com/NVIDIA/Megatron-LM/blob/main/megatron/arguments.py


In [ ]:
%cd Large-scale-lm-eng-tutorial/notebooks/

<br>

## 3. Parallelformers

<img src="../images/parallelformers.png" width=360>

So far, we have trained models using Megatron-LM. Although Megatron-LM provides excellent Tensor Parallelism capabilities, it could not parallelize models that were trained using the Hugging Face `transformers` library, which we commonly use. To address this limitation, TUNiB released an open-source project called `parallelformers` in 2021. `parallelformers` is a tool that allows Tensor Parallelism to be applied to inference for most models trained with Hugging Face `transformers` using just one or two lines of code.

Let’s install `parallelformers`.


In [ ]:
!pip install parallelformers

`parallelformers` can parallelize an existing model using the `parallelize` function as shown in the code below, and it also provides several additional options such as `num_gpus` and `fp16`.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from parallelformers import parallelize

if __name__ == "__main__":
    model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B")
    tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")
    parallelize(model, num_gpus=4, fp16=True, verbose="simple")

    inputs = tokenizer(
        "Parallelformers is",
        return_tensors="pt",
    )

    outputs = model.generate(
        **inputs,
        num_beams=5,
        no_repeat_ngram_size=4,
        max_length=15,
    )

    print(f"\nOutput: {tokenizer.batch_decode(outputs)[0]}")


Note: `parallelformers` uses shared memory for inter-process data communication. Therefore, **when using it in environments that allow only limited resources, such as docker, you must increase the shared memory size.**

You can increase the shared memory size using the `docker run ... --shm_size=?gb` option, or remove the shared memory limitation using the `docker run ... --ipc=host` option. It has been confirmed that almost all issues occurring in docker are caused by shared memory limitations, and using larger models requires allocating a larger amount of shared memory.


In [ ]:
!python ../src/parallelformers_inference.py

### How Parallelformers Works

<br>

![](../images/tensor_replace.png)

<br>

Then how is `parallelformers` able to perform Tensor parallelism without modifying the model code? The answer lies in the `Tensor Replacement` mechanism. `parallelformers` first extracts all parameters from the original model, then splits the tensors in the same way as Megatron-LM, and replaces the original parameters in the model with the partitioned tensors. By doing so, it can achieve parallelization without changing the model structure. Using this approach, approximately 70 different models were able to be parallelized. Although several additional mechanisms were introduced, they are not directly related to tensor parallelism and are therefore omitted here. If you would like more detailed information, please refer to the following links.

- Korean: https://tunib.notion.site/TECH-2021-07-26-Parallelformers-_-0dcceeaddc5247429745ba36c6549fe5
- English: https://tunib.notion.site/TECH-2021-07-26-Parallelformers-Journey-to-deploying-big-models_TUNiB-32b19a599c38497abaad2a98727f6dc8
